# DSCI 100 Project Final Report - Group 15

## Using Player Behavioural Data to Predict Newsletter Subscriptions

Chloe Bayley, Ray Wu, William Tees-DeBeyer

## Introduction

For this data science project, a Computer Science research group at UBC, led by Frank Wood, has set up a Minecraft (a popular video game) server to collect data on player behaviour to study how people play video games. In particular, the researchers collected data on the players' ages, gender, experience level, subscription status to the game-related newsletter, and the total number of hours played on the server. This led us to question: **Can the age, playtime, gender and experience of a user be used to predict whether or not they will subscribe to a game-related newsletter?**

To explore this question, we will be using the players.csv dataeset, which contains the following variables: 
- experience: character variable that describes the experience level of a user.
- subscribe: logical variable that describes whether that user is subscribed to the newsletter (TRUE) or not (FALSE).
- hashedEmail: encrypted email of the user.
- played_hours: the total number of hours the user has on the MineCraft server.
- name: name of the user
- gender: categorical variable describing the gender of the user
- Age: numerical variable describinig the age of the user.

**Note**: For this project, hashedEmail and names are not relevant variables, and will be filtered out.


In [ ]:
# Libraries used/ Run before everything else!
library(tidyverse)
library(repr)
library(tidymodels)

In [ ]:
players_data <- read_csv("players.csv")